In [1]:
import pandas as pd
import spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
train_data = pd.read_csv("/content/drive/MyDrive/dataset/word2vec/dataset/treino.csv")
train_data.sample(5)

,title,text,date,category,subcategory,link
61963,Brasileiro Thiago Monteiro é vice-campeão do C...,"O cearense Thiago Monteiro, 22, foi derrotado ...",2016-12-06,esporte,NaN,http://www1.folha.uol.com.br/esporte/2016/06/1...
48010,Bate-boca entre senadores interrompe tramitaçã...,Senadores bateram boca e quase entraram em con...,2017-05-23,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/05/1...
34897,Filme mostra o homem Pasolini por trás da égid...,O encontro entre Pier Paolo Pasolini e Abel Fe...,2015-06-11,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2015/11...
89973,São muitos os cenários para eventual desfecho ...,CHÂTEAU BNDES Um conhecedor de palácios e cas...,2017-09-07,colunas,eliogaspari,http://www1.folha.uol.com.br/colunas/eliogaspa...
23410,São Paulo desiste de jogador que se declarou c...,"O jogador Getterson, do J. Malucelli, nem ao m...",2016-06-22,esporte,NaN,http://www1.folha.uol.com.br/esporte/2016/06/1...


In [3]:
#execute just once
#!python -m spacy download pt_core_news_sm

In [4]:
nlp = spacy.load("pt_core_news_sm")

In [5]:
text = "Rio de Janeiro é uma cidade maravilhosa"
doc = nlp(text)
doc

Rio de Janeiro é uma cidade maravilhosa

In [6]:
type(doc[2])

spacy.tokens.token.Token

In [7]:
texts_to_treatment = (titles.lower() for titles in train_data["title"])

In [8]:
def treat_texts(doc):
    valid_tokens = []
    for token in doc:
        is_valid = not token.is_stop and token.is_alpha
        if is_valid:
            valid_tokens.append(token.text)

    if len(valid_tokens) > 2:
        return  " ".join(valid_tokens)

text = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(text)
treat_texts(doc)

'Rio Janeiro cidade maravilhosa'

In [9]:
text = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(text)
treat_texts(doc)

'Rio Janeiro cidade maravilhosa'

In [10]:
from time import time

t0 = time()
treated_texts = [treat_texts(doc) for doc in nlp.pipe(texts_to_treatment,
                                                        batch_size = 1000,
                                                        n_process = -1)]
tf = time() - t0
print(tf/60)

2.7994814991950987


In [11]:
treated_titles = pd.DataFrame({"titulo": treated_texts})
treated_titles.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


In [12]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

In [13]:
print(len(treated_titles))
treated_titles = treated_titles.dropna().drop_duplicates()
print(len(treated_titles))

90000
84466


In [14]:
list_tokens_list = [title.split(" ") for title in treated_titles.titulo]

In [15]:
import logging

logging.basicConfig(format="%(asctime)s : - %(message)s", level = logging.INFO)

w2v_model = Word2Vec(sg = 0,
                      window = 2,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_model.build_vocab(list_tokens_list, progress_per=5000)

In [16]:
dir(w2v_model)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_input_data_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_job_params',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_minimize_model',
 '_raw_word_count',
 '_save_specials',
 '_set_train_params',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_update_job_params',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'accuracy',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'ca

In [17]:
w2v_model.corpus_count

84466

In [18]:
w2v_model.train(list_tokens_list, 
                 total_examples=w2v_model.corpus_count,
                 epochs = 30)

(14584753, 16207260)

In [19]:
w2v_model.wv.most_similar("google")

[('apple', 0.5821973085403442),
 ('facebook', 0.5461843013763428),
 ('amazon', 0.5033813714981079),
 ('airbnb', 0.4901963770389557),
 ('fbi', 0.4821164011955261),
 ('yahoo', 0.4692976474761963),
 ('alibaba', 0.4662802815437317),
 ('unilever', 0.4657129645347595),
 ('software', 0.46326470375061035),
 ('uber', 0.4617651402950287)]

In [20]:
w2v_model.wv.most_similar("microsoft")

[('tesla', 0.5957540273666382),
 ('amazon', 0.5926369428634644),
 ('unilever', 0.5600866675376892),
 ('lego', 0.5420367121696472),
 ('walmart', 0.5257389545440674),
 ('viajante', 0.5246310234069824),
 ('canais', 0.524333119392395),
 ('chrysler', 0.5148504376411438),
 ('linkedin', 0.5089726448059082),
 ('braskem', 0.5071768760681152)]

In [21]:
w2v_model.wv.most_similar("barcelona")

[('barça', 0.6008591651916504),
 ('madrid', 0.5784553289413452),
 ('chelsea', 0.5743868350982666),
 ('juventus', 0.5562494993209839),
 ('bayern', 0.5439050197601318),
 ('lazio', 0.5386303067207336),
 ('psg', 0.5381754636764526),
 ('leicester', 0.5328654646873474),
 ('munique', 0.5298463106155396),
 ('united', 0.5284828543663025)]

In [22]:
w2v_model.wv.most_similar("messi")

[('suárez', 0.5805963277816772),
 ('cristiano', 0.5658708810806274),
 ('cavani', 0.5483490228652954),
 ('benzema', 0.5212342739105225),
 ('ronaldo', 0.5172131061553955),
 ('tevez', 0.5084702968597412),
 ('barça', 0.5066540241241455),
 ('neymar', 0.5000272393226624),
 ('chuteiras', 0.49200013279914856),
 ('barcelona', 0.4894178509712219)]

In [23]:
w2v_model.wv.most_similar("gm")

[('chrysler', 0.6805555820465088),
 ('embraer', 0.6704780459403992),
 ('volks', 0.6356314420700073),
 ('braskem', 0.6339007616043091),
 ('fiat', 0.623374342918396),
 ('honda', 0.6215111017227173),
 ('tesla', 0.6074561476707458),
 ('toyota', 0.6062800884246826),
 ('csn', 0.6039988398551941),
 ('renault', 0.5838131904602051)]

In [24]:
#model Skip-Gram training
w2v_model_sg = Word2Vec(sg = 1,
                      window = 5,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_model_sg.build_vocab(list_tokens_list, progress_per=5000)

w2v_model_sg.train(list_tokens_list, 
                 total_examples=w2v_model_sg.corpus_count,
                 epochs = 30)

(14583677, 16207260)

In [25]:
w2v_model_sg.wv.most_similar("google")

[('reguladores', 0.4313902258872986),
 ('buffett', 0.41028472781181335),
 ('android', 0.410110741853714),
 ('yahoo', 0.40221139788627625),
 ('apple', 0.39857929944992065),
 ('waze', 0.39305540919303894),
 ('patentes', 0.3902628421783447),
 ('facebook', 0.38974088430404663),
 ('app', 0.37017232179641724),
 ('concorda', 0.36158061027526855)]

In [26]:
w2v_model.wv.most_similar("google")

[('apple', 0.5821973085403442),
 ('facebook', 0.5461843013763428),
 ('amazon', 0.5033813714981079),
 ('airbnb', 0.4901963770389557),
 ('fbi', 0.4821164011955261),
 ('yahoo', 0.4692976474761963),
 ('alibaba', 0.4662802815437317),
 ('unilever', 0.4657129645347595),
 ('software', 0.46326470375061035),
 ('uber', 0.4617651402950287)]

In [27]:
w2v_model_sg.wv.most_similar("gm")

[('metalúrgicos', 0.5778361558914185),
 ('motors', 0.5573704838752747),
 ('honda', 0.5174570679664612),
 ('audi', 0.5006668567657471),
 ('fiat', 0.4873671233654022),
 ('airbag', 0.4816415309906006),
 ('chrysler', 0.4815548360347748),
 ('airbags', 0.4788525402545929),
 ('montadora', 0.47723719477653503),
 ('autoguiados', 0.47298765182495117)]

In [28]:
w2v_model.wv.most_similar("gm")

[('chrysler', 0.6805555820465088),
 ('embraer', 0.6704780459403992),
 ('volks', 0.6356314420700073),
 ('braskem', 0.6339007616043091),
 ('fiat', 0.623374342918396),
 ('honda', 0.6215111017227173),
 ('tesla', 0.6074561476707458),
 ('toyota', 0.6062800884246826),
 ('csn', 0.6039988398551941),
 ('renault', 0.5838131904602051)]

In [29]:
w2v_model.wv.save_word2vec_format("/content/drive/MyDrive/dataset/word2vec/dataset/modelo_cbow.txt", binary=False)
w2v_model_sg.wv.save_word2vec_format("/content/drive/MyDrive/dataset/word2vec/dataset/modelo_skipgram.txt", binary=False)